# Testing in Python

This notebook covers fundamental aspects of **Testing** in Python:

1. **Unit Testing**
2. **Integration Testing**
3. **Mocking**

Testing ensures our code works as intended, remains stable as it evolves, and catches regressions or bugs early in the development process.

## Table of Contents
1. [Unit Testing](#unit-testing)
2. [Integration Testing](#integration-testing)
3. [Mocking](#mocking)

We'll include code examples using Python's built-in **unittest** framework along with some notes on **pytest** where relevant.

## 1. Unit Testing <a name="unit-testing"></a>

### What is Unit Testing?
Unit tests focus on **individual units** of code—often functions or methods. Each test aims to verify that one small "unit" works correctly in isolation.

**Key Points**:
- Each test typically checks a single logical aspect (e.g., function input -> expected output).
- Unit tests are fast, isolated, and form the foundation of the testing pyramid.

### Example Project Setup
Consider a Python module `math_utils.py` with a few simple functions:

```python
# math_utils.py

def add(a, b):
    return a + b

def divide(a, b):
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b
```

We want to create a test module `test_math_utils.py` to verify these functions' correctness.


In [4]:
import unittest

# For demonstration in this notebook, let's define math_utils here
def add(a, b):
    return a + b

def divide(a, b):
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

class TestMathUtils(unittest.TestCase):

    def test_add_positive_numbers(self):
        self.assertEqual(add(2, 3), 5)
        self.assertEqual(add(0, 10), 10)

    def test_add_negative_numbers(self):
        self.assertEqual(add(-1, -2), -3)
        self.assertEqual(add(-5, 5), 0)

    def test_divide_normal_cases(self):
        self.assertAlmostEqual(divide(10, 2), 5.0)
        self.assertAlmostEqual(divide(-10, 2), -5.0)

    def test_divide_by_zero(self):
        with self.assertRaises(ValueError):
            divide(5, 0)

# We can run tests programmatically:
if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.003s

OK


**Explanation**:
- We define a test class (`TestMathUtils`) inheriting from `unittest.TestCase`.
- Each method starting with `test_` is automatically recognized as a test.
- We use **assertions** like `assertEqual` to check expected vs. actual behavior.
- `with self.assertRaises(...)` checks that a specific exception is raised.
- Normally, you'd keep tests in a separate file from your main code.


## 2. Integration Testing <a name="integration-testing"></a>

**Integration Tests** verify that multiple components or modules work together as expected. They may test:
- Interactions between functions, classes, or modules.
- External services (APIs, databases).
- The flow of data through multiple layers of an application.

### Example Scenario
Imagine we have a function that reads data from one module, processes it with another, and returns combined results. For demonstration, we’ll just pretend to integrate two simple modules (though typically you'd see more complexity).

In [8]:
import random

def fetch_data():
    """
    Pretend this function fetches data from an external source.
    We'll mock this out in tests.
    """
    return [random.randint(1, 100) for _ in range(5)]

def process_data(data_list):
    """
    Simple function that returns (sum, avg).
    """
    if not data_list:
        return 0, 0
    total = sum(data_list)
    avg = total / len(data_list)
    return total, avg

def combined_workflow():
    """
    Integration function that fetches data and then processes it.
    """
    data = fetch_data()
    return process_data(data)

# Let's test them in an "integration" sense.
import unittest

class TestIntegration(unittest.TestCase):

    def test_integration_normal(self):
        """Test the entire workflow without mocking"""
        total, avg = combined_workflow()
        # We can't predict exact random values, but we can check ranges.
        self.assertTrue(0 < total <= 500)  # 5 random ints, each up to 100
        self.assertTrue(0 < avg <= 100)

if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.005s

OK


**Explanation**:
- `combined_workflow()` calls both `fetch_data()` and `process_data()`. This is an **integration** because multiple parts are being used together.
- **In production** or more complex scenarios, you might integrate with databases, message queues, or external APIs here.
- The test checks that the result is within expected bounds.


## 3. Mocking <a name="mocking"></a>

**Mocking** is the practice of replacing real objects with "fake" ones (mocks) that simulate their behavior. This is critical when:
- You want **isolation** (e.g., no real network calls, no real database writes).
- External dependencies are slow, unavailable, or expensive.
- Reproducible tests with predictable results.

### Python Tools for Mocking
- `unittest.mock` (built into Python)
- `pytest-mock` (for pytest users)

Let’s see how to mock the `fetch_data()` function so our integration test doesn’t rely on random values.

In [12]:
from unittest.mock import patch
import unittest

# We'll reuse the same combined_workflow, but mock fetch_data.

class TestMocking(unittest.TestCase):

    @patch('__main__.fetch_data', return_value=[10, 20, 30, 40, 50])
    def test_mocked_workflow(self, mock_fetch):
        total, avg = combined_workflow()
        # Because we know the returned list is [10, 20, 30, 40, 50],
        # we can check precise results.
        self.assertEqual(total, 150)
        self.assertEqual(avg, 30)

if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)

......
----------------------------------------------------------------------
Ran 6 tests in 0.005s

OK


**Explanation**:
- We use `@patch('__main__.fetch_data', return_value=[10, 20, 30, 40, 50])` to replace `fetch_data()` with a mock returning `[10, 20, 30, 40, 50]`.
- Now our test is **deterministic**—we know exactly what data is used.
- This is **unit-testing** or **integration-testing** with mocks to isolate external dependencies.

In real scenarios, you'd mock out **network calls**, **database queries**, or **file I/O** to keep tests fast and deterministic.

# Conclusion

In this notebook, we've covered:
1. **Unit Testing**: Testing individual functions using `unittest`.
2. **Integration Testing**: Ensuring multiple pieces work together, possibly including external services.
3. **Mocking**: Replacing real objects/functions with mocks for isolation and deterministic tests.

## Further Suggestions
- Explore **pytest** for a more concise testing syntax.
- Use **coverage** tools (like `coverage.py`) to measure how much of your code is tested.
- Integrate your tests into CI/CD pipelines to run automatically on each commit.

With robust testing, you can develop more confidently, knowing that your code remains stable and correct as it evolves.